In [4]:
from datamodule.preprocessing import *
from model.metric import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import pandas as pd
import polars as pl

train = pl.read_csv('data/train.csv')[:100]
sanity_check = pd.read_csv('data/train_features.csv')
train.shape

(100, 3)

In [6]:
train_feature_df = create_features(train)
train_feature_df.head()

,essay_id,score,word_1_cnt,word_2_cnt,word_3_cnt,word_4_cnt,word_5_cnt,word_6_cnt,word_7_cnt,word_8_cnt,...,tfidf_2236_y,tfidf_2237_y,tfidf_2238_y,tfidf_2239_y,tfidf_2240_y,tfidf_2241_y,tfidf_2242_y,tfidf_2243_y,tfidf_2244_y,tfidf_2245_y
0,000d118,3,491,471,391,289,169,119,76,46,...,0,0,0,0,0,0,0,0,0,0
1,000fe60,3,332,311,248,184,103,63,41,29,...,0,0,0,1,1,0,0,0,0,0
2,001ab80,4,550,539,445,328,200,135,103,72,...,0,0,2,0,0,0,0,0,0,0
3,001bdc0,4,446,431,348,277,217,175,133,85,...,0,0,0,0,0,4,4,0,0,0
4,002ba53,3,373,354,280,213,170,138,112,68,...,0,0,0,0,0,0,0,0,0,0


In [14]:
import numpy as np

feature_names = list(filter(lambda x: x not in ['full_text', 'essay_id','score'], train_feature_df.columns))

X = train_feature_df[feature_names].astype(np.float32).values
y = train_feature_df['score'].astype(np.float32).values - a
labels = train_feature_df['score'].astype(int).values

In [7]:
metrics = {
    'f1': (lambda x, y: x + y)
}

metrics['f1'](1, 2)

3

In [15]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import f1_score

def feature_select_wrapper():
    models = []
    predictions = []
    f1_scores = []
    kappa_scores = []

    # Part 1.
    print('feature_select_wrapper...')
    features = feature_names

    straified_k_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    fse = pd.Series(0, index=features)
         
    for train_indices, validation_indices in straified_k_folds.split(X, labels):
        X_train, X_validation = X[train_indices], X[validation_indices]
        y_train, y_validation = y[train_indices], y[validation_indices], 
        labels_validation = labels[validation_indices]

        lgbm_regressor = lgb.LGBMRegressor(
            objective=quadratic_weight_kappa_objective,
            metrics='None',
            learning_rate=0.05,
            max_depth=5,
            num_leaves=10,
            colsample_bytree=0.3,
            reg_alpha=0.7,
            reg_lambda=0.1,
            n_estimators=700,
            random_state=412,
            extra_trees=True,
            class_weight='balanced',
            verbosity=-1
        )

        model = lgbm_regressor.fit(
            X_train,
            y_train,
            eval_names=['train', 'validation'],
            eval_set=[(X_train, y_train), (X_validation, y_validation)],
            eval_metric=quadratic_weighted_kappa_metric,
        )

        models.append(model)
        predictions_validation = model.predict(X_validation)
        predictions_validation = predictions_validation + a
        predictions_validation = predictions_validation.clip(1, 6).round()
        predictions.append(predictions_validation)
        f1 = f1_score(labels_validation, predictions_validation, average='weighted')
        f1_scores.append(f1)

        kappa = cohen_kappa_score(labels_validation, predictions_validation, weights='quadratic')
        kappa_scores.append(kappa)

    
        print(f'F1 score across fold: {f1}')
        print(f'Cohen kappa score across fold: {kappa}')

        fse += pd.Series(model.feature_importances_, features)  
    
    # Part 4.
    feature_select = fse.sort_values(ascending=False).index.tolist()[:13000]
    print('done')
    return feature_select

feature_select = feature_select_wrapper()

feature_select_wrapper...


/home/dminn/projects/kaggle-competitions/automatic-essay-scoring-2/.venv/lib/python3.12/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[LightGBM] [Info] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with 

In [16]:
len(feature_select)

13000